<a href="https://colab.research.google.com/github/fatehirfan1/ai-meme-recommender/blob/main/AI_Meme_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q torch torchvision pillow git+https://github.com/openai/CLIP.git gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00


In [4]:
import os, torch, clip, numpy as np, gradio as gr
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
MEME_PATH = "/content/drive/MyDrive/meme_project/memes"

**Loading CLIP Model**

In [5]:
# Loading Clip Model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
print("CLIP loaded")

100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 317MiB/s]


CLIP loaded


**Load and Encode**

In [6]:
# Load and Encode Images
print("Encoding meme images...")
image_files = [f for f in os.listdir(MEME_PATH) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

image_embs = []
for img_name in image_files:
    path = os.path.join(MEME_PATH, img_name)
    img = preprocess(Image.open(path).convert("RGB")).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = clip_model.encode_image(img).cpu().numpy()
    image_embs.append(emb)

image_embs = np.vstack(image_embs)
print(f"Loaded {len(image_files)} memes")

Encoding meme images...
Loaded 100 memes


**Recommendation Function:**

In [7]:
# Meme recommendation function
def recommend_meme(user_text):
    txt_emb = clip_model.encode_text(clip.tokenize(user_text).to(device)).cpu().detach().numpy()
    sims = cosine_similarity(txt_emb, image_embs)[0]
    best_idx = np.argmax(sims)
    best_meme = image_files[best_idx]
    return Image.open(os.path.join(MEME_PATH, best_meme))

**UI:**


In [8]:
# UI
ui = gr.Interface(
    fn=recommend_meme,
    inputs=gr.Textbox(label="Enter your situation", placeholder="e.g. When you study all night and still fail..."),
    outputs=gr.Image(label="Best Matching Meme"),
    title="AI Meme Recommender ",
    description="Type your situation and AI will find the best meme for it using CLIP."
)

ui.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b24e63af5d16ade02d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
